In [1]:
import os
import random
import numpy as np
import cv2
import json
import pickle
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from tqdm.notebook import tqdm

## Settings

In [2]:
# set the size of the dataset, manually
samples = 10000
# set the target orientation
orientation = 1
# set the scale for resizing the images
scale = .5

## Helper Functions

In [3]:
# function for reading the Json files
def idx_finder(folder):
    '''
    This function takes the folder name (string),
    and returns a list of indices of the target
    orientation and valid frames in the folder.
    '''
    # set the specific folder's path
    path = 'Data_MIT/' + folder

    # read the json files
    # ------------------------------------------------------------------
    # read the Face Crop json
    with open(path + '/appleFace.json', 'r') as file:
        face = json.load(file)
    # read the Left Eye json
    with open(path + '/appleLeftEye.json', 'r') as file:
        eye_l = json.load(file)
    # read the Right Eye json
    with open(path + '/appleRightEye.json', 'r') as file:
        eye_r = json.load(file)
    # read the Scree json
    with open(path + '/screen.json', 'r') as file:
        screen = json.load(file)
    
    # make a list of all valid indices
    # ------------------------------------------------------------------
    # list of all frames with full face in it
    faces = [idx for idx, val in enumerate(face['IsValid']) if val == 1]
    # list of all frames with the left eye in it
    eyes_l = [idx for idx, val in enumerate(eye_l['IsValid']) if val == 1]
    # list of all frames with the right eye in it
    eyes_r = [idx for idx, val in enumerate(eye_r['IsValid']) if val == 1]
    # list of all portrait frames
    portraits = [idx for idx, ori in enumerate(screen['Orientation']) if ori == orientation]
    # intersection of all those lists
    indices = [idx for idx in faces if idx in eyes_l and idx in eyes_r and idx in portraits]
    
    return indices
    
    
    
# function for getting the gaze coordinates
def get_labels(folder, indices):
    '''
    The function takes the folder name (string),
    and a list of indices (list) and returns an
    np.array of gaze coordinates or the labels.
    '''
    # set the specific folder's path
    path = 'Data_MIT/' + folder
    
    # read the dot json
    with open(path + '/dotInfo.json', 'r') as file:
        dot = json.load(file)
    
    # list of XCam coordinate for valid frames
    dot_x = [x for idx, x in enumerate(dot['XCam']) if idx in indices]
    # list of YCam coordinate for valid frames
    dot_y = [y for idx, y in enumerate(dot['YCam']) if idx in indices]
    # make an np.array out of the coordinates for valid frames
    dots = np.column_stack((dot_x, dot_y))
    
    return dots



# function for read, resize, and save an image
def resize(path, scale):
    '''
    The function takes a path to an image (string),
    and a scale (float) to rescale the image and
    save it.
    '''
    # read the image
    img = cv2.imread(path)
    # resize the image
    new_h = int(img.shape[1]*scale)
    new_w = int(img.shape[0]*scale)
    img = cv2.resize(img, (new_h, new_w))
    # count the number of files already available in the destination folder
    files = len(os.listdir('Data_MTCNN/'))
    # set the path and name for the file
    save_path = 'Data_MTCNN/' + '0'*(5-len(str(files))) + str(files) + '.jpg'
    # and save the image
    cv2.imwrite(save_path, img);
    
    

## Create the Dataset

In [4]:
# make a list of the folders in the data directory
folders = [name for name in os.listdir('Data_MIT/')]
# drop the irrelevant file in the data directory
folders.remove('LICENSE.md')

# create a list of valid and portrait frames in folders
frames_all = []
for folder in tqdm(folders):
    # make a list of valid frames
    indices = idx_finder(folder)
    frames = [folder+str(idx) for idx in indices]
    frames_all = frames_all + frames
    
# pick the 'samples' number of the frames, randomly
# and in case of 'samples' larger than the number of
# all frames, pick them all
frames = random.sample(frames_all, k=min(samples, len(frames_all)))

# set a list for labels
labels = []

# make a list of the folders picked and listed in frames
folder_list = sorted(list(set([frame[:5] for frame in frames])))

# for each folder...
for folder in tqdm(folder_list):
    # get indices of picked frames in that folder
    indices = [int(frame[5:]) for frame in frames if frame[:5] == folder]
    # and then returns a list of the labels and append it to labels list
    labels.append(get_labels(folder, indices))
    

    # for each file in the folder...
    for idx in indices:
        # set the path to the file
        path = 'Data_MIT/' + folder + '/frames/' + '0'*(5-len(str(idx))) + str(idx) + '.jpg'
        # read, resize, and save the file
        resize(path, scale)


## Save the Dataset

In [6]:
# convert the labels list to an array
labels = np.array(labels)
# and save it in a pickle file
with open('Data_MTCNN/labels.pkl','wb') as f:
    pickle.dump(labels, f)



# read images and convert them to an np.array
images = []

for i in tqdm(range(samples)):
    # read the image
    img = plt.imread('Data_MTCNN/' + '0'*(5-len(str(i))) + str(i) + '.jpg')
    # append the image to the images list
    images.append(img)

# convert the labels list to an array
images = np.array(images)
# and save it in a pickle file
with open('Data_MTCNN/images.pkl','wb') as f:
    pickle.dump(images, f)



# save the frames list for replication purposes
with open('Data_MTCNN/frames.pkl','wb') as f:
    pickle.dump(frames, f)
